# LSTMusic with key and metro

In [1]:
import pandas as pd
import re
from music21 import *
import numpy as np

import time
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Data Formatting

In [2]:
f = open("../data/jiggs.txt", "r")
raw_input = f.read()

In [3]:
class Repertoir():
    def __init__(self, path):
        self.path = path
        f = open(path, "r")
        self.string = f.read()
        self.handler = abcFormat.ABCHandler()
        self.handler.process(self.string)
        self.songs_handlers = self.handler.splitByReferenceNumber()
        self.songs = {}
        self.__process()
    
    def __str__(self):
        return self.string
    
    
    def __process(self):
        for ref_number, handler in self.songs_handlers.items():
            self.songs[ref_number] = Song(handler)
            
    def get_part_vocab(self):
        tokens = []
        for ref_number, song in self.songs.items():
            tokens+= song.part
        tokens = list(set(tokens))            
        return tokens
    
    def get_metadata_vocab(self, key):
        tokens = []
        for ref_number, song in self.songs.items():
            tokens+= [song.metadata[key]]
        tokens = list(set(tokens))            
        return tokens    

In [4]:
class Song():
    def __init__(self, handler):
        self.handler = handler
        self.metadata = {
            'X':1,
            'T':'Unknown',
            'S':'Unknown',
            'M':'none',
            'L':'',
            'Q':'',
            'K':''
        }
        self.part = []
        self.__process()
        
    def __process(self):
        for token in self.handler.tokens:
            meta_data_ended=False
            if isinstance(token, abcFormat.ABCMetadata):
                if token.tag in self.metadata.keys():
                    if self.metadata[token.tag]=='' or not meta_data_ended:
                        self.metadata[token.tag] = token.data
                else:
                    self.metadata[token.tag] = token.data
            elif isinstance(token, abcFormat.ABCNote ) or isinstance(token, abcFormat.ABCBar):
                meta_data_ended = True
                self.part.append(token.src)
    
    def __str__(self):
        return self.to_abc()
    
    def to_abc(self):
        output = ''
        for key, value in self.metadata.items():
            output+= key+':'+value+"\n"
        for note in self.part:
            output+=note
        return output

In [5]:
def generate_char_idx_mappings(vocab):
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    return char2idx, idx2char

In [6]:
def get_input_tensors(part, k, m, part_char2idx, k_char2idx, m_char2idx):
    part_tensor = torch.tensor([part_char2idx[note] for note in part[0:-1]], dtype=torch.long)
    k_tensor = torch.tensor([k_char2idx[k] for note in part[0:-1]], dtype=torch.long)
    m_tensor = torch.tensor([m_char2idx[m] for note in part[0:-1]], dtype=torch.long)
    return part_tensor, k_tensor, m_tensor,

def get_target_tensor(part, part_char2idx):
    target_tensor = torch.tensor([part_char2idx[note] for note in part[1:]], dtype=torch.long)
    return target_tensor

In [7]:
rep = Repertoir('../data/jiggs.txt')

In [8]:
part_vocab = rep.get_part_vocab()
m_vocab = rep.get_metadata_vocab('M')
k_vocab = rep.get_metadata_vocab('K')

In [9]:
part_char2idx, part_idx2char = generate_char_idx_mappings(part_vocab)
k_char2idx, k_idx2char = generate_char_idx_mappings(k_vocab)
m_char2idx, m_idx2char = generate_char_idx_mappings(m_vocab)

## LSTMusic

In [10]:
class LSTMusic(nn.Module):

    def __init__(self, part_embedding_dim, k_embedding_dim, m_embedding_dim, lstm_dim, part_vocab_size, k_vocab_size, m_vocab_size):
        super(LSTMusic, self).__init__()
        self.lstm_dim = lstm_dim
        
        self.part_embeddings = nn.Embedding(part_vocab_size, part_embedding_dim)
        self.k_embeddings = nn.Embedding(k_vocab_size, k_embedding_dim)
        self.m_embeddings = nn.Embedding(m_vocab_size, m_embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(m_embedding_dim+k_embedding_dim+part_embedding_dim, lstm_dim)

        # The linear layer that maps from hidden state space to tag space
        self.dense = nn.Linear(lstm_dim, part_vocab_size)

    def forward(self, part, k, m, prev_state):
        part_embeds = self.part_embeddings(part)
        k_embeds = self.k_embeddings(k)
        m_embeds = self.m_embeddings(m)
        combined = torch.cat((m_embeds, k_embeds, part_embeds), 1)
        lstm_out, state = self.lstm(combined.view(len(part), 1, -1), prev_state)
        output = self.dense(lstm_out.view(len(part), -1))
        return output, state
    
    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_dim),
                torch.zeros(1, batch_size, self.lstm_dim))

In [28]:
def generate(model, part, k, m, length):
    model = model.eval()
    notes = []
    with torch.no_grad():  # no need to track history in sampling
        state_h, state_c = model.zero_state(1)
        for note in part:
            note_tensor, k_tensor, m_tensor = get_input_tensors([note,' '], k, m, part_char2idx, k_char2idx, m_char2idx)
            output, (state_h, state_c) = model(note_tensor, k_tensor, m_tensor, (state_h, state_c))
        _, top_ix = torch.topk(output[0], k=5)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        notes.append(part_idx2char[choice])
    for _ in range(length):
        note_tensor = torch.tensor([choice])
        output, (state_h, state_c) = model(note_tensor, k_tensor, m_tensor, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=5)
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        notes.append(part_idx2char[choice])

    abc = "M:{}\nK:{}\n".format(m,k) + ''.join(part) + ''.join(notes)
    print(abc)
    return abc, notes

In [29]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [34]:
part_embedding_dim = 128
k_embedding_dim = 16
m_embedding_dim = 16
lstm_dim = 256
part_vocab_size = len(part_vocab)
k_vocab_size = len(k_vocab)
m_vocab_size = len(m_vocab)

nb_epoch = 10000
lr = 0.001
max_norm = 5

start_part = rep.songs[1].part[0:25]
start_k = rep.songs[1].metadata['K']
start_m = rep.songs[1].metadata['M']
length = 100

In [35]:
model = LSTMusic(part_embedding_dim, k_embedding_dim, m_embedding_dim, lstm_dim, part_vocab_size, k_vocab_size, m_vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

start = time.time()
epoch = 0
while True:
    epoch+=1
    print('\n========= Epoch {} out of {} ({} %) =========\n'.format(epoch, nb_epoch, (epoch)/nb_epoch*100))
    nb_iters = len(rep.songs)
    iteration = 0
    print_every = nb_iters//10
    state_h, state_c = model.zero_state(1)

    for ref_num, song in rep.songs.items():
        
        iteration+=1
        
        model = model.train()
        
        optimizer.zero_grad()
        
        part, k, m  = get_input_tensors(song.part, song.metadata['K'], song.metadata['M'], part_char2idx, k_char2idx, m_char2idx)
        target = get_target_tensor(song.part, part_char2idx)
        
        output, (state_h, state_c) = model(part, k, m, (state_h, state_c))
        
        state_h = state_h.detach()
        state_c = state_c.detach()
        
        loss = criterion(output, target)
        
        loss.backward()
        
        _ = torch.nn.utils.clip_grad_norm_(
                model.parameters(), max_norm)
        
        optimizer.step()
        
        if iteration % print_every == 0:
            print('%s (%d %d%%) %.4f' % (timeSince(start), iteration, iteration / nb_iters * 100, loss.item()))
    print('\nGenerating example :\n')
    abc, op_notes = generate(model, start_part, start_k, start_m, length)    
    torch.save(model.state_dict(),'../models/model-{}.pth'.format(epoch))


========= Epoch 1 out of 10000 (0.01 %) =========

0m 5s (34 10%) 4.5244
0m 11s (68 20%) 3.4713
0m 16s (102 30%) 3.8422
0m 22s (136 40%) 3.2834
0m 28s (170 50%) 2.8605
0m 33s (204 60%) 3.6109
0m 39s (238 70%) 3.4549
0m 45s (272 80%) 2.8883
0m 50s (306 90%) 3.2168
0m 56s (340 100%) 3.3478

Generating example :

M:6/8
K:D
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2f"G7"G6" ""e"d=e"Dm"g"G""Bm"bg"A/c+"d2"G""Bm"b"G7""f"d3"Gm"g2"Bm7"f"D""d"F"E7"d3"D"b2"Bm"f3^c"C"E2"Dm/f"f2"Em"E6g2"A7"A2"E"B"E/dim"g2"Gm"f2"A7/e"f2"Em"g3/2^c"C"d2"Am"=c"c#"b"A7"e3"c#"F2"F"d3"C"B"A7"[f2A2]"D7/a"A"D/f+"a3"B7"e2"Am"e3e"F#m"a2"A"e3/2"Em"F2a3/2"A7"=g2"C"^F3"D"D"C"B"Bm"A2"C7"dG3[GAc]"D"[F3A3d3]"C7"c2"C"G3"e"d"G7"B2"Am"a"A7"e3/2e6c'2"E"E"C"E2"D/f+"A^e3"G""Bm"b"G7"^A2"Bb"D2"C"a3"G"d3/2^G"D""f#"f2z3"D7"d6"Em"E6"Bm"d"A/c+"d2"D"a3/2=g"E7"^G"C7"c2"A7"G"E7"f2"F7"c2"Em"E6"C""e"g"D7"e3"Bm"b2"D7"d3"Em"g3/2"A"a2"C"B"F"f"C/e"e"D"=f"Gm"f2_B"D"^f"C""e"G"A7"[f2A2]"D7"A2

========= Epoch 2 out of 10000 (0.02 %) =========

1m 2s (

9m 24s (34 10%) 1.5444
9m 30s (68 20%) 1.5325
9m 37s (102 30%) 1.5795
9m 44s (136 40%) 0.7933
9m 52s (170 50%) 1.3961
9m 59s (204 60%) 1.9295
10m 6s (238 70%) 1.7340
10m 13s (272 80%) 1.4704
10m 20s (306 90%) 1.4628
10m 27s (340 100%) 2.0979

Generating example :

M:6/8
K:D
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2f"G""Bm"b"G"A2"A7/g"A3a/2"Em"g/2=D"Gm"f2"D"D"D""Bm"F3" ""d#"F"B7"e"F"c3"G7""f"d3"B7"FC"A7"F"A7""A7"e_aB/2"A7"a"C"=c3"Eb"G3"Bm"d3"D7"f2"(A7)"F2A3"A7/e"f"D"G"A7"d2"Dm"e2"D7"F2"A7"e3"Dm"a3"Am""F"c"F#7"e"A"G"G""3"B"C"C6"F#m"F"E7"=B"G/b"G3"C"E2"F""c"A2"Am"c2"F"c3"C"g2"G7"G6G3e/2"eb"g"D""Bm"f2"3"c"Dm/f"f2" ""d"B,2"G7"G6"Dm"d2"A7""A7"e"Em"B3"E"f3"Em"F[2"D"A2"E"d"A7"=g2"Em"f3"G"c"A7/e"f"D"F"Bb"_b2"F"a3"G/b"g3"c#"F2"A7"A2"A"B"Am"a2"Am"e3B,6"C"a3"D""Bm"F3"Bb7"D"C""e"g"Gm"B"D7"f"c#"F2B2"Dm"B2"Bm"b2"E7"^G"Em"ga3/2"Em"E6"B7"F2"Eb"G3"F#m"a2B2"D"A3"E7"F"Em"f3"F#7"d"D/f+"a3"A"b

========= Epoch 11 out of 10000 (0.11 %) =========

10m 34s (34 10%) 1.3772
10m 41s (68 20%) 1.4664
10m 46s (1

19m 0s (34 10%) 0.5122
19m 6s (68 20%) 0.8916
19m 12s (102 30%) 0.7822
19m 18s (136 40%) 0.3516
19m 23s (170 50%) 0.8769
19m 29s (204 60%) 1.1396
19m 34s (238 70%) 0.9709
19m 40s (272 80%) 0.9919
19m 46s (306 90%) 0.6397
19m 52s (340 100%) 0.9819

Generating example :

M:6/8
K:D
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2f"C/e"g3"C"E"B7"E2f"(E7)"e3"C7"e"D""a"A"B7"f"Dm"B"G7"G2"c#"F2"G"D3^D"Bb"B3c'/2"D""A"a3"Dm"F"Dm"f2"G"G3/2"C"d2C/2"E7/b9"=f2"A"e/2"A7"e2c'/2^F2"D"[F3A3d3]"F"A2"A"c3B/2"A""F#m"a2"A7"[e3-c3-][E3c3]"D"f3/2"D7"d3/2"E7""F#m"G3"Bb""f"f"Em"b2"C""F"e"Dm"a2"Am""e"c2"Eb"G3"Cm"g"G/b"g"B7"b"D/f+"A"Em"B,"Am"c2"G"e2F3"D7"A2"C"=G"F#7"c2"Am"g3"D"ag2"C"C6"D""Bm"d3"C"G"E7"^G"Am""c"e"E7"g2"D"a2"A"E3"C"c"G"f"D7"e2"C7"c2"G7/f"d2"d#"fg/2"(E7)"e2"G7"B2"E"d2"G/b"g2"G"g3"Bm"c"Gm"ac'/2"Bm"d3C3C3"A"g"Eb"G3"G7"G"A7"A2"E"e2"D7/a"A"Gm"g"Gm"g"Bm"B3"C"G3C/2"E7"A2"G"G3/2^D"Am"a"C7"e3"B7"E2"d#"A"3"B

========= Epoch 20 out of 10000 (0.2 %) =========

19m 58s (34 10%) 0.4528
20m 5s (68 20%) 0.9025
20m 1

28m 32s (34 10%) 0.2441
28m 39s (68 20%) 0.5377
28m 45s (102 30%) 0.3864
28m 52s (136 40%) 0.1417
28m 58s (170 50%) 0.5405
29m 4s (204 60%) 0.6984
29m 9s (238 70%) 0.5299
29m 15s (272 80%) 0.6049
29m 21s (306 90%) 0.3432
29m 26s (340 100%) 0.4363

Generating example :

M:6/8
K:D
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2f"Em"g/2"Em"g/2"Bm"F3"eb"g"C/e"e"B7"e2"G7"B2"C"g2c'/2"A/c+"A"C7"g"Dm"B^a"E"A"F#m"A2"D7"f2"Bm"ae/2"G/b"g"G7"B2"F"B2"Gm"d"A7"[f2A2]"F"d3"D"a"C"C6"E7"d3"eb"ga3/2"Bm"B2"C""e"c2"G""d"B3"A7"a"E7""b"g"G/b"G3B2"F"f6"A/c+"A"D"f3/2"A/c+"d2f3"Bm"D2"F"A2"G""G"b"E7"g3"(A7)"F2"A7"c"D7"f2"D7"d6_a"G"f"Em"B3"D7/a"d"A7"E3=c3e"F"d3"Em/g"g2"Bm"b2"d#"Af3_B=B2"Bm"D"A7"e"B7"F2"A7"[e3-c3-]"e"E"A"G"A7/e"c2"Em""C"e/2"A"f2"Am"=c=F"Em"b2"E7"^G3a3/2"Am"A6"D7"e3"E7"c"C"E"C/e"e=f"F7"e=c3"F"f3"F7"_e2"A7"e2"C"C6"C7"c2" ""d#"F"D7/a"d"B7"f^f2"G/b"b2"D/f+"a2"D"D2"G7"b3"A"A3/2"Em"f3"F7"_E3

========= Epoch 29 out of 10000 (0.29 %) =========

29m 32s (34 10%) 0.2766
29m 38s (68 20%) 0.5327
29m 43s (102 3

38m 0s (34 10%) 0.1573
38m 6s (68 20%) 0.3427
38m 11s (102 30%) 0.1685
38m 17s (136 40%) 0.0857
38m 23s (170 50%) 0.2773
38m 30s (204 60%) 0.3201
38m 36s (238 70%) 0.3518
38m 43s (272 80%) 0.3446
38m 50s (306 90%) 0.1942
38m 57s (340 100%) 0.2434

Generating example :

M:6/8
K:D
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2f"A7"d"E7"=F"G7"G2"D"fa3"D7"e3[E3c3]"Am""c"eA,2c2"E7"b3"Eb"G3"C"C3"F"a"Em"c2"E"f3"D"[F2A2B2]"(E7)"e3"G7"=B"d#"^D"D"[F3A3d3]"Gm"G3"G/b"g2"D"b2"D7"f" ""d"B,2"E7"=B"E7"c2"A"b"A"B"G/b"d"Cm"e"D"B"D""f#"a^G"Bb"B,"D"=f"D""d"F"Am"c"C"=g"D/f+"a3"F"c'2"Gm"B3"A"A3/2"A7"G3=B2a3"B7"f^f2F3"Dm"e2"D7/a"d"E"c2"C"C6[GAc]"B7"F"A/e"g2"A7"E3"C"A6=F"E7""b"g"Em"B"Em"B2"G/b"g2"D""Bm"F3"Em"G"Bm/f+"d2d2c'/2" ""B7"a^c/2" ""d#"F"F#m"Fb/2"Dm"=f"C"d2"Em"G2"G"F"e"E"D"[F2A2B2]"Am"g3"Bm"e2"D7"f^G2"C"c3/2"Am/c"c2"E7/b9"=f2"A"A"D"F3"G"f2"C7"B"Eb"G3"G"e3"D7"d3/2"Bm/f+"d2"Gm"c3"B7"f"Dm"c2"C""e"E2"Bm"A"G7"b3

========= Epoch 38 out of 10000 (0.38 %) =========

39m 4s (34 10%) 0.1317
39m 11s (68 20%) 0.26

...

250m 32s (34 10%) 0.0376
250m 38s (68 20%) 0.0497
250m 44s (102 30%) 0.0753
250m 51s (136 40%) 0.0089
250m 58s (170 50%) 0.0380
251m 5s (204 60%) 0.0379
251m 11s (238 70%) 0.0379
251m 19s (272 80%) 0.0376
251m 25s (306 90%) 0.0524
251m 33s (340 100%) 0.0371

Generating example :

M:6/8
K:D
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2f"C""e"g2"F"a"C""e"g2"B7"E2=B"Em"E6"D"^f"D7/a"c2"G7"A"Gm"G3"Am"f/2"G7"G"D/a"A2^F2"D""Bm"d3"G7"G2"A7"^c3"G"A2"F"c3"C"a2"Am/c"b"Bb"D2"Bm"d3"A/e"g2"Bm/a"B2"Em"G"D"~d3"G7"c2_e"B7"e2"D7"G"Bm"e2"D"g2"D7"f2"E7"^G3^e3" ""d"B,2"D""a"A"G7"f"Bm"d"G"A2"A"E3"A7"E"G7""f"d3"Dm"=f"D"g2"E7"F"A"A3/2"Bm"c2"D/f+"f2"Bm"a2"D7"f"Bb""f"f"c#"b"A7"e2" ""d"B,2"D/c+"d2"G"d"D/f+"d2"G"c"A7"G"G"a2"Am"d2a/2"f#"g"B7"f"Gm"G3"D"[F3A3d3]"g"d2"D"b2"C"c"G"b3/2"E7"g3"Dm"B2"Bm"e3"Em"g"f"=F3"G7"B2"A7"^c3"C"=c3"Dm"a3"Am""e"c2a3/2"A"d"A7/e"g2"Em"c"C"A6"A7"e3"A"A3"A"G2"Bm"e"f"=F3"D7/a"A"G7"d'2"E"B"C"g"G/b"d"G7"G2"F"a"d#"A"C"G2

========= Epoch 236 out of 10000 (2.36 %) =========

251m 40s (34 10%) 0

260m 24s (34 10%) 0.0013
260m 31s (68 20%) 0.0038
260m 38s (102 30%) 0.0015
260m 45s (136 40%) 0.0033
260m 52s (170 50%) 0.0018
260m 58s (204 60%) 0.0020
261m 4s (238 70%) 0.0023
261m 10s (272 80%) 0.0023
261m 16s (306 90%) 0.0013
261m 21s (340 100%) 0.0012

Generating example :

M:6/8
K:D
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2f[GAc]"E/dim"g2"D7/a"c2"C"C3"Am"d2=F"D/a"A2"A7/g"A3"Bb"B,"G7"=B"Bm"d2"B7"e"(E7)"e3C"Bb"_b2"Dm"d2"D"d3"C"g3"Am"G"Bm"B2c'/2"Eb"A"F#m""A"c3"A"cfCa3"F#m/a"c2"Em"a2"g"B=Ae2"B7"F2"C/e"G"E7"d3"C"a3b/2"F"f3"Em"E"D/c+"d2"E7"F"D7"f"Em"F"G/b"g"G7"f"D/a"A2f2"D/a"A2^D"Bm"F3"A7"G"E7"=F"Am"f2"A7/e"c2=A"Am""F"c"G"g3"E7"^G"Bm"e"Am"c"3"e"F#m/a"c2"D"[F3A3d3]"e"d"D""a"d"B7"F2"D"A,2"3"e[e2c2]"A7/g"A3"B7"f"G"g2"A"e/2"A"G2"D"a"Gm"B3"C"B"E7"D2G2"F"=F3"A"A"Em"g"G/b"G3"A7/e"f"F"a3"Am"a"Bm7"f2"Dm"=fA/2"G""Em"G3"Gm"g[FAd]"G/b"b2"D7/a"d"B7"A3A3"C""e"g2"Bm"f3"C"a3A3"D7/a"A

========= Epoch 245 out of 10000 (2.45 %) =========

261m 27s (34 10%) 0.0011
261m 33s (68 20%) 0.0046
261m 39s (

275m 33s (34 10%) 0.0024
275m 48s (68 20%) 0.0040
276m 1s (102 30%) 0.0035
276m 15s (136 40%) 0.0314
276m 29s (170 50%) 0.0027
276m 41s (204 60%) 0.0045
276m 53s (238 70%) 0.0030
277m 7s (272 80%) 0.0032
277m 19s (306 90%) 0.0014
277m 37s (340 100%) 0.0022

Generating example :

M:6/8
K:D
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2f"Bm7"f"A"E2"F#m"f"E7"B,"Em"g"D7"E2"F"F2"A7/g"A3"C"c2"Am"c"C"f"c#"F2"D"a2"F#m"A3"G""Bm"b"Gm"B^e2"Dm"G"B7"A"B7/f+"f"Em"E6"Bm"e^e3"E7/b9"=f3^D"A7"b"D""Bm"d3A3/2"G"g3"F#7"e"D7"=c"B7/f+"f"C"e/2"D7"AA2"G"G4"D7/a"c2"Dm"a2"D/f+"d2"A7"=g2"c#"f"D"a2"C"f"E7"E2"D"a2"G"d"A7"F2B2"G"A3z3"G/b"g3"F#7"ea/2"3"Ba3/2"E/dim"g2"E"B2"F#7"e2"Am"d2"F7"AB2"G/b"d"C"G"A"A3b/2"A7/e"a2"E"^G2" ""g"c" ""D"f"F#7"e"C/e"e"G""Bm"b"Em""(C)"e2"A7"d"C7"A"D7"C"D7"A"C"C2"Em"g"Em"E"Am"G"Cm"g"c#"f"Am"G"A"G2"C"E2"F"a"Dm"f2"E/dim"g2d'3/2"B"e"C""e"G"B7"a2"A7/e"f"E"c2e2"B7"E2"Dm"g"Em"c2"Bm"d2"C""e"g

========= Epoch 254 out of 10000 (2.54 %) =========

277m 51s (34 10%) 0.0019
278m 5s (68 20%) 0.0029
27

294m 45s (34 10%) 0.1226
294m 59s (68 20%) 0.3976
295m 11s (102 30%) 0.2348
295m 25s (136 40%) 0.0900
295m 38s (170 50%) 0.2029
295m 51s (204 60%) 0.4044
296m 3s (238 70%) 0.1837
296m 16s (272 80%) 0.3089
296m 29s (306 90%) 0.2657
296m 42s (340 100%) 0.2288

Generating example :

M:6/8
K:D
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2fz3"c#"b"Em""C"e/2"G"G3/2"Bm"da3"G7"B"Gm"g2"G7"d"Bm7"f=B"Em"g/2"D""Bm"F3"A7"[e3-c3-]"G/b"d_e"Am"d2"Bm"D2"A/c+"A"G/b"g3"C""F"e"E"e"E7"d2"D7"A2"g"d2"E"E"G/b"g2"Dm"e"E/dim"g2"C/dim"c"Dm""f"A2" ""f/="B"g"BD6"A7"[e3-c3-]"Eb"G3"F"a"C"E2"Em"d3=e"A7"[f2A2]a3"A"g"C7"A"F#m"e2"A"A3"A7/e"g2"G/b"d"E"^G"C""F"e"Em"F|"C"=g=c3g/2"F#m"e"C""e"g" ""c"e2"D7"E2" ""D"f"Dm""f"A2"B7"^d"Gm"g"A7"E3"D"d/2"F"a"G7"b3"Am"c"G/b"G3"G""d"F2"Cdim""eb"g3"C"C6"C7"c2"C"c2"Eb"e"G/b"B2"A"[A3c3]"A"c3"E7"b"b"c" ""gb"e"G7"=B"G7/d"E2d6"A"E3"C/e"G2"F"A2"D7"E"D7"e3"F7"_E3c2"B7"F2" ""g"c"C"A6"D"a3/2:|"G""g"B2"D"D2e/2A2=g

========= Epoch 263 out of 10000 (2.63 %) =========

296m 57s (34 10%) 0.1448
297

KeyboardInterrupt: 